# Poisson modeling using ARIMA models

In [ ]:
import math
import numpy
import matplotlib.pyplot as plt
from src.poisson import Poisson
from src.arima import ARIMA
from src.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error
import statsmodels.api as sm
import itertools
import pylab
%matplotlib inline

### Example of our Poisson process

In [ ]:
p = Poisson()
example, _ = p.generate(6)

plt.figure(figsize=(20,7))
plt.plot(example)

### Apply the ARIMA model to a new data set that is more varied

In [ ]:
a1,_ = p.generate(14)
a2,_ = p.generate(3,2)
a3,_ = p.generate(8)
a4,_ = p.generate(2,2)
actual = numpy.concatenate((a1,a2,a3,a4))

arima = ARIMA()
pred = arima.predict(actual)

### Plot the prediction

In [ ]:
actual = actual[71:-2]
error = math.sqrt(mean_squared_error(pred, actual))

plt.figure(figsize=(20,7))
plt.title('RMSE: {:.2f}'.format(error))
plt.plot(actual)
plt.plot(pred)

### Evaluate the residual

In [ ]:
residual = []
for i in range(len(actual)):
    val = actual[i] - pred[i][0]
    residual.append(val)

plt.figure(figsize=(20,7))
plt.plot(residual)

That visually looks very far from normal.

### Draw the qq-plot of this data

In [ ]:
sm.qqplot(numpy.array(residual), line='45')
pylab.show()

Pretty bad!

### Use the normed Poisson and look at the residual

In [ ]:
_, a1 = p.generate(14)
_, a2 = p.generate(3,2)
_, a3 = p.generate(8)
_, a4 = p.generate(2,2)
actual = numpy.concatenate((a1,a2,a3,a4))

arima = ARIMA()
pred = arima.predict(actual)

actual = actual[71:-2]
error = math.sqrt(mean_squared_error(pred, actual))

residual = []
for i in range(len(actual)):
    val = actual[i] - pred[i][0]
    residual.append(val)

### Plot the residual

In [ ]:
plt.figure(figsize=(20,7))
plt.plot(residual)

Looks a little better.

### Draw the qq-plot of the normed residual

In [ ]:
sm.qqplot(numpy.array(residual), line='45')
pylab.show()

But it is not!

### Let's try a wide swatch of models including seasonality, scored by AIC

In [ ]:
p = Poisson()
_, a1 = p.generate(14)
_, a2 = p.generate(3,2)
_, a3 = p.generate(8)
_, a4 = p.generate(2,2)
actual = numpy.concatenate((a1,a2,a3,a4))

max_order = 3
period = 24

p = d = q = range(0, max_order)
orders = list(itertools.product(p,d,q))

results = {}
for order in orders:
    for season in [(x[0], x[1], x[2], period) for x in orders]:
        try:
            mod = sm.tsa.statespace.SARIMAX(
                actual,
                order                 = order,
                seasonal_order        = season,
                enforce_stationarity  = False,
                enforce_invertibility = False
            )
            res = mod.fit()
            results[res.aic] = 'ARIMA {} x {}'.format(order, season)
            print('ARIMA {} x {}: {}'.format(order, season, res.aic))
        except:
            continue

s = sorted(results.iterkeys())[0]
print('\n{}: {}'.format(results[s], s))

This turns out to be ARIMA (0, 0, 2) x (1, 1, 2, 24).  Let's try it.

In [ ]:
p = Poisson()

_, a1 = p.generate(14)
_, a2 = p.generate(3,2)
_, a3 = p.generate(8)
_, a4 = p.generate(2,2)
actual = numpy.concatenate((a1,a2,a3,a4))

sarimax = SARIMAX()
pred = sarimax.predict(actual)

In [ ]:
actual = actual[71:-2]
error = math.sqrt(mean_squared_error(pred, actual))

residual = []
for i in range(len(actual)):
    val = actual[i] - pred[i][0]
    residual.append(val)

In [ ]:
plt.figure(figsize=(20,7))
plt.plot(residual)

In [ ]:
sm.qqplot(numpy.array(residual), line='45')
pylab.show()

### General results

The data we have here does not admit a particularly good solution with ARIMA methods.  Assuming the integral term as a means of addressing seasonality makes a big impact.  However, there are no choices of coefficient that result in a residual that is normal.  The ARIMA model does have a fairly faithful tracking of the fluctuating data because it is coupled to the data.  It tends to be better the more of the history of the function the coefficients are computed from.  For some problems this may be an adequate solution, but it should be well known that the approach leaves a lot to be desired.  Even norming the Poisson data by the mean, while changing the visual character of the residual, does not significantly alter the qq-plot.